In [21]:
# %%bash
set -e
sudo apt-get update -y
sudo apt-get install -y git curl gnupg software-properties-common unzip python3-pip
echo "✅  Base packages installed"


Hit:1 https://apt.releases.hashicorp.com jammy InRelease
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]      
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease                         
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease    
Fetched 257 kB in 3s (91.8 kB/s)
Reading package lists... 0%Reading package lists... 0%Reading package lists... 0%Reading package lists... 4%Reading package lists... 4%Reading package lists... 4%Reading package lists... 4%Reading package lists... 11%Reading package lists... 27%Reading package lists... 44%Reading package lists... 45%Reading package lists... 45%Reading package lists... 46%Reading package lists... 46%Reading package lists... 55%Reading package lists... 55%Reading package lists... 59%Reading package lists... 68%Reading package lists... 68%Reading package lists... 73%Reading package lists... 73%Reading package lists... 73%

In [22]:
# %%bash
set -e
if ! command -v terraform &>/dev/null; then
  curl -fsSL https://apt.releases.hashicorp.com/gpg | \
       sudo gpg --dearmor -o /usr/share/keyrings/hashicorp.gpg
  echo "deb [signed-by=/usr/share/keyrings/hashicorp.gpg] https://apt.releases.hashicorp.com $(lsb_release -cs) main" | \
       sudo tee /etc/apt/sources.list.d/hashicorp.list > /dev/null
  sudo apt-get update -y
  sudo apt-get install -y terraform
fi
terraform -version


Terraform v1.11.4
on linux_amd64


In [5]:
pwd

/work


In [12]:
# %%bash
# ─── EDIT THESE FOUR LINES ────────────────────────────────────────────────────
export GIT_REPO="https://github.com/Akkey01/MLops-Final-Project.git"   # ◀︎ EDIT
export CLONE_DIR="/work/MLops-Final-Project"                           # ◀︎ EDIT
export OPENRC_FILE="/home/ng3230_nyu_edu/work/MLops-Final-Project/infrastructure/tf/kvm/openrc.sh"                             # ◀︎ EDIT
export TF_SUFFIX="team39"                                              # ◀︎ EDIT
export TF_KEYPAIR="id_rsa_chameleon"
# ──────────────────────────────────────────────────────────────────────────────
echo "Variables set:"
printf " GIT_REPO     = %s\n CLONE_DIR   = %s\n OPENRC_FILE = %s\n TF_SUFFIX   = %s\n TF_KEYPAIR  = %s\n" \
       "$GIT_REPO" "$CLONE_DIR" "$OPENRC_FILE" "$TF_SUFFIX" "$TF_KEYPAIR"


Variables set:
 GIT_REPO     = https://github.com/Akkey01/MLops-Final-Project.git
 CLONE_DIR   = /work/MLops-Final-Project
 OPENRC_FILE = /home/ng3230_nyu_edu/work/MLops-Final-Project/infrastructure/tf/kvm/openrc.sh
 TF_SUFFIX   = team39
 TF_KEYPAIR  = id_rsa_chameleon


In [4]:
sudo apt-get install -y git python3-pip
pip install --user --upgrade python-openstackclient


Reading package lists... 0%Reading package lists... 0%Reading package lists... 0%Reading package lists... 4%Reading package lists... 4%Reading package lists... 4%Reading package lists... 4%Reading package lists... 12%Reading package lists... 29%Reading package lists... 45%Reading package lists... 45%Reading package lists... 46%Reading package lists... 46%Reading package lists... 46%Reading package lists... 55%Reading package lists... 55%Reading package lists... 65%Reading package lists... 68%Reading package lists... 68%Reading package lists... 73%Reading package lists... 73%Reading package lists... 73%Reading package lists... 73%Reading package lists... 73%Reading package lists... 73%Reading package lists... 73%Reading package lists... 73%Reading package lists... 82%Reading package lists... 82%Reading package lists... 85%Reading package lists... 94%Reading package lists... 94%Reading package lists... 98%Reading package lists... 98%Reading package lists... 98%Reading package lists... 98

In [13]:
# %%bash
set -e
pip install --user --upgrade --no-cache-dir python-openstackclient
if ! grep -q '.local/bin' ~/.bashrc; then
  echo 'export PATH=$HOME/.local/bin:$PATH' >> ~/.bashrc
fi
export PATH=$HOME/.local/bin:$PATH
openstack --version


openstack 8.0.0


In [14]:
# %%bash
set -e
if [ ! -d "$CLONE_DIR/.git" ]; then
  git clone --recurse-submodules "$GIT_REPO" "$CLONE_DIR"
else
  echo "Repo exists; doing git pull ..."
  git -C "$CLONE_DIR" pull --ff-only
fi


Repo exists; doing git pull ...
Already up to date.


In [15]:
# %%bash
set -e
if [ ! -f "$OPENRC_FILE" ]; then
  echo "ERROR: $OPENRC_FILE not found" >&2
  exit 1
fi
unset $(env | grep -oE '^OS_[A-Z_]+')    # clean old vars
source "$OPENRC_FILE"
openstack token issue >/dev/null && echo "✅  Auth OK"


✅  Auth OK


In [16]:
# %%bash
set -e
cd "$CLONE_DIR/infrastructure/tf/kvm"

# generate a local tfvars file so commands stay short
cat > local.auto.tfvars <<EOF
suffix = "${TF_SUFFIX}"
key    = "${TF_KEYPAIR}"
EOF

terraform init -input=false
terraform plan -out apply.plan
terraform apply -auto-approve apply.plan

echo "🎉  Terraform finished. Floating IPs:"
# terraform output -json public_ips | jq -r 'to_entries[] | "\(.key): \(.value)"'


Initializing the backend...
Initializing provider plugins...
- Reusing previous version of terraform-provider-openstack/openstack from the dependency lock file
- Using previously-installed terraform-provider-openstack/openstack v1.51.1

Terraform has been successfully initialized!

You may now begin working with Terraform. Try running "terraform plan" to see
any changes that are required for your infrastructure. All Terraform commands
should now work.

If you ever set or change modules or backend configuration for Terraform,
rerun this command to reinitialize your working directory. If you forget, other
commands will detect it and remind you to do so if necessary.
data.openstack_networking_secgroup_v2.allow_8080: Reading...
data.openstack_networking_secgroup_v2.allow_9090: Reading...
data.openstack_networking_subnet_v2.sharednet2_subnet: Reading...
data.openstack_networking_secgroup_v2.allow_http_80: Reading...
data.openstack_networking_secgroup_v2.allow_8081: Reading...
data.openstack

In [18]:
echo "🎉  Terraform finished. Floating IP:"
terraform output -raw floating_ip_out     # prints 129.114.26.17


🎉  Terraform finished. Floating IP:
╷
│ Warning: No outputs found
│ 
│ The state file either has no outputs defined, or all the defined outputs are
│ empty. Please define an output in your configuration with the `output`
│ keyword and run `terraform refresh` for it to become available. If you are
│ using interpolation, please verify the interpolated value is not empty. You
│ can use the `terraform console` command to assist.
╵


In [23]:
#!/usr/bin/env bash
set -euo pipefail

##############################################################################
# CONFIG — edit paths only if your repo layout differs
##############################################################################
TF_DIR="/work/MLops-Final-Project/infrastructure/tf/kvm"
ANSIBLE_DIR="/work/MLops-Final-Project/infrastructure/ansible"
SSH_KEY="$HOME/.ssh/id_rsa_chameleon"
ANSIBLE_USER="cc"
##############################################################################

echo -e "\n>>> 1. Ensure Ansible is installed"
if ! command -v ansible-playbook &>/dev/null; then
  pip install --user --upgrade --no-cache-dir ansible
  export PATH="$HOME/.local/bin:$PATH"
fi
ansible --version | head -1

echo -e "\n>>> 2. Read node1 floating-IP from Terraform output"
cd "$TF_DIR"
CONTROL_IP=$(terraform output -raw floating_ip_out)
echo "Control node (node1) public IP: $CONTROL_IP"

echo -e "\n>>> 3. Build dynamic inventory with ProxyCommand"
cat > "$ANSIBLE_DIR/inventory.ini" <<EOF
[control]
node1 ansible_host=${CONTROL_IP}

[workers]
node2 ansible_host=192.168.1.12
node3 ansible_host=192.168.1.13

[all:vars]
ansible_user=${ANSIBLE_USER}
ansible_ssh_private_key_file=${SSH_KEY}
# let Ansible jump through node1 to reach the private IPs of node2/3
ansible_ssh_common_args=-o ProxyCommand="ssh -i ${SSH_KEY} -W %h:%p ${ANSIBLE_USER}@${CONTROL_IP}"
EOF

cat "$ANSIBLE_DIR/inventory.ini"

echo -e "\n>>> 4. Connectivity test (ansible ping)"
ANSIBLE_HOST_KEY_CHECKING=False \
  ansible -i "$ANSIBLE_DIR/inventory.ini" all -m ping

echo -e "\n>>> 5. Run playbooks in sequence"
cd "$ANSIBLE_DIR"

run_playbook() {
  local pb=$1
  echo -e "\n─── Running $pb ───"
  ansible-playbook -i inventory.ini "$pb"
}

run_playbook general.yml
run_playbook pre_k8s.yml
run_playbook k8s/cluster.yml
run_playbook post_k8s.yml
run_playbook argocd/add_platform.yml
run_playbook argocd/add_staging.yml
run_playbook argocd/add_prod.yml
run_playbook argocd/add_canary.yml
run_playbook argocd/workflow_templates_apply.yml

echo -e "\n✅  All playbooks finished successfully!"



>>> 1. Ensure Ansible is installed
ansible 2.10.17

>>> 2. Read node1 floating-IP from Terraform output
Control node (node1) public IP: 129.114.26.17

>>> 3. Build dynamic inventory with ProxyCommand
[control]
node1 ansible_host=129.114.26.17

[workers]
node2 ansible_host=192.168.1.12
node3 ansible_host=192.168.1.13

[all:vars]
ansible_user=cc
ansible_ssh_private_key_file=/home/ng3230_nyu_edu/.ssh/id_rsa_chameleon
# let Ansible jump through node1 to reach the private IPs of node2/3
ansible_ssh_common_args=-o ProxyCommand="ssh -i /home/ng3230_nyu_edu/.ssh/id_rsa_chameleon -W %h:%p cc@129.114.26.17"

>>> 4. Connectivity test (ansible ping)
The authenticity of host '129.114.26.17 (129.114.26.17)' can't be established.
ED25519 key fingerprint is SHA256:+iK7b/21KchejBShrxnkhk/ZZiKNNdQGgum7au92R1c.
This key is not known by any other names
The authenticity of host '129.114.26.17 (129.114.26.17)' can't be established.
ED25519 key fingerprint is SHA256:+iK7b/21KchejBShrxnkhk/ZZiKNNdQGgum7au92R

In [24]:
ls -l ~/.ssh
ssh-add -l           # shows keys loaded in your agent (if any)


total 8
-rw------- 1 ng3230_nyu_edu users 978 May  8 23:38 known_hosts
-rw-r--r-- 1 ng3230_nyu_edu users 142 May  8 23:38 known_hosts.old
4096 SHA256:L0dwiEJVmJE7VmwD10x1KZadRj+6hTZgUjvVrfJKiVw /work/.ssh/id_rsa (RSA)


In [34]:
# %%bash
CONTROL_IP=129.114.26.17                # ← leave as is
SSH_KEY=~/.ssh/id_rsa

cat > /work/MLops-Final-Project/infrastructure/ansible/inventory.ini <<EOF
[control]
node1 ansible_host=${CONTROL_IP}

[workers]
node2 ansible_host=192.168.1.12
node3 ansible_host=192.168.1.13

[all:vars]
ansible_user=cc
ansible_ssh_private_key_file=${SSH_KEY}
ansible_ssh_common_args='-o StrictHostKeyChecking=no'

[workers:vars]
ansible_ssh_common_args='-o StrictHostKeyChecking=no -o ProxyCommand="ssh -i ${SSH_KEY} -o StrictHostKeyChecking=no -W %h:%p cc@${CONTROL_IP}"'
EOF

cat /work/MLops-Final-Project/infrastructure/ansible/inventory.ini


[control]
node1 ansible_host=129.114.26.17

[workers]
node2 ansible_host=192.168.1.12
node3 ansible_host=192.168.1.13

[all:vars]
ansible_user=cc
ansible_ssh_private_key_file=/home/ng3230_nyu_edu/.ssh/id_rsa
ansible_ssh_common_args='-o StrictHostKeyChecking=no'

[workers:vars]
ansible_ssh_common_args='-o StrictHostKeyChecking=no -o ProxyCommand="ssh -i /home/ng3230_nyu_edu/.ssh/id_rsa -o StrictHostKeyChecking=no -W %h:%p cc@129.114.26.17"'


In [41]:
# %%bash
ANSIBLE_DIR=/work/MLops-Final-Project/infrastructure/ansible
export PATH=$HOME/.local/bin:$PATH
ANSIBLE_HOST_KEY_CHECKING=False

echo "Installing python3-six, python3-setuptools on all nodes …"
ansible -i $ANSIBLE_DIR/inventory.ini all \
  -m raw \
  -a 'sudo DEBIAN_FRONTEND=noninteractive apt-get update -y &&
      sudo DEBIAN_FRONTEND=noninteractive apt-get install -y python3-six python3-setuptools' \
  --become -vv


Installing python3-six, python3-setuptools on all nodes …
ansible 2.10.17
  config file = None
  configured module search path = ['/home/ng3230_nyu_edu/.ansible/plugins/modules', '/usr/share/ansible/plugins/modules']
  ansible python module location = /opt/conda/lib/python3.10/site-packages/ansible
  executable location = /opt/conda/bin/ansible
  python version = 3.10.9 | packaged by conda-forge | (main, Feb  2 2023, 20:20:04) [GCC 11.3.0]
No config file found; using defaults
Skipping callback 'default', as we already have a stdout callback.
Skipping callback 'minimal', as we already have a stdout callback.
Skipping callback 'oneline', as we already have a stdout callback.
META: ran handlers
node3 | CHANGED | rc=0 >>
Get:1 http://nova.clouds.archive.ubuntu.com/ubuntu noble InRelease [256 kB]
Hit:2 http://security.ubuntu.com/ubuntu noble-security InRelease               
Hit:3 http://nova.clouds.archive.ubuntu.com/ubuntu noble-updates InRelease     
Hit:4 http://nova.clouds.archive.ubun

In [39]:
# %%bash
ANSIBLE_DIR=/work/MLops-Final-Project/infrastructure/ansible
export PATH=$HOME/.local/bin:$PATH
ANSIBLE_HOST_KEY_CHECKING=False

ansible -i $ANSIBLE_DIR/inventory.ini all \
  -m raw \
  -a "sudo apt-get update -y && sudo apt-get install -y python3 python3-apt python3-pip" \
  --become


node1 | CHANGED | rc=0 >>
Get:1 http://nova.clouds.archive.ubuntu.com/ubuntu noble InRelease [256 kB]
Get:2 http://nova.clouds.archive.ubuntu.com/ubuntu noble-updates InRelease [126 kB]
Get:3 http://security.ubuntu.com/ubuntu noble-security InRelease [126 kB]
Get:4 http://nova.clouds.archive.ubuntu.com/ubuntu noble-backports InRelease [126 kB]
Get:5 http://nova.clouds.archive.ubuntu.com/ubuntu noble-updates/main amd64 Packages [1,066 kB]
Get:6 http://nova.clouds.archive.ubuntu.com/ubuntu noble-updates/main amd64 Components [161 kB]
Get:7 http://nova.clouds.archive.ubuntu.com/ubuntu noble-updates/main amd64 c-n-f Metadata [13.5 kB]
Get:8 http://nova.clouds.archive.ubuntu.com/ubuntu noble-updates/universe amd64 Packages [1,062 kB]
Get:9 http://security.ubuntu.com/ubuntu noble-security/main amd64 Packages [820 kB]
Get:10 http://nova.clouds.archive.ubuntu.com/ubuntu noble-updates/universe amd64 Components [376 kB]
Get:11 http://nova.clouds.archive.ubuntu.com/ubuntu noble-updates/universe a

In [51]:
# %%bash
CFG_DIR=/work/MLops-Final-Project/infrastructure/ansible
mkdir -p "$CFG_DIR"

cat > "$CFG_DIR/ansible.cfg" <<'EOF'
[defaults]
inventory            = ./inventory.ini
remote_user          = cc
private_key_file     = ~/.ssh/id_rsa
host_key_checking    = False
retry_files_enabled  = False
stdout_callback      = yaml
any_errors_fatal     = False

[ssh_connection]

ssh_args = -o StrictHostKeyChecking=off -o UserKnownHostsFile=/dev/null -o ForwardAgent=yes -o ProxyCommand="ssh -o StrictHostKeyChecking=no -o UserKnownHostsFile=/dev/null -W %h:%p cc@129.114.26.17"
pipelining = True
EOF

echo "Created $CFG_DIR/ansible.cfg:"
cat "$CFG_DIR/ansible.cfg"


Created /work/MLops-Final-Project/infrastructure/ansible/ansible.cfg:
[defaults]
inventory            = ./inventory.ini
remote_user          = cc
private_key_file     = ~/.ssh/id_rsa
host_key_checking    = False
retry_files_enabled  = False
stdout_callback      = yaml
any_errors_fatal     = False

[ssh_connection]

ssh_args = -o StrictHostKeyChecking=off -o UserKnownHostsFile=/dev/null -o ForwardAgent=yes -o ProxyCommand="ssh -o StrictHostKeyChecking=no -o UserKnownHostsFile=/dev/null -W %h:%p cc@129.114.26.17"
pipelining = True


In [52]:
# ansible -i $ANSIBLE_DIR/inventory.ini all -m ping
export ANSIBLE_CONFIG=/work/MLops-Final-Project/infrastructure/ansible/ansible.cfg
ansible all -m ping



An exception occurred during task execution. To see the full traceback, use -vvv. The error was: ModuleNotFoundError: No module named 'ansible.module_utils.six.moves'
node1 | FAILED! => {
    "ansible_facts": {
        "discovered_interpreter_python": "/usr/bin/python3"
    },
    "changed": false,
    "module_stderr": "Warning: Permanently added '129.114.26.17' (ED25519) to the list of known hosts.\r\nWarning: Permanently added '129.114.26.17' (ED25519) to the list of known hosts.\r\nConnection to 129.114.26.17 closed.\r\n",
    "module_stdout": "Traceback (most recent call last):\r\n  File \"/home/cc/.ansible/tmp/ansible-tmp-1746751408.3651714-6390-168724267906021/AnsiballZ_ping.py\", line 102, in <module>\r\n    _ansiballz_main()\r\n  File \"/home/cc/.ansible/tmp/ansible-tmp-1746751408.3651714-6390-168724267906021/AnsiballZ_ping.py\", line 94, in _ansiballz_main\r\n    invoke_module(zipped_mod, temp_path, ANSIBALLZ_PARAMS)\r\n  File \"/home/cc/.ansible/tmp/ansible-tmp-1746751408.365

: 2

In [53]:
#!/usr/bin/env bash
set -euo pipefail

##########################################################################
# EDIT THESE PATHS / VARIABLES ONLY IF NEEDED
##########################################################################
TF_DIR="/work/MLops-Final-Project/infrastructure/tf/kvm"
ANSIBLE_DIR="/work/MLops-Final-Project/infrastructure/ansible"
SSH_KEY="$HOME/.ssh/id_rsa"            # path to the private key that works
ANSIBLE_USER="cc"                      # default CC username
##########################################################################

echo -e "\n>>> 1. Install/upgrade Ansible locally (if missing)"
if ! command -v ansible-playbook &>/dev/null; then
  pip install --user --upgrade --no-cache-dir ansible
  export PATH="$HOME/.local/bin:$PATH"
fi
ansible --version | head -1

echo -e "\n>>> 2. Fetch node1 floating IP from Terraform output"
cd "$TF_DIR"
CONTROL_FIP=$(terraform output -raw floating_ip_out)
echo "Control node public IP: $CONTROL_FIP"

echo -e "\n>>> 3. Build inventory.ini (with ProxyCommand for workers)"
cat > "$ANSIBLE_DIR/inventory.ini" <<EOF
[control]
node1 ansible_host=${CONTROL_FIP}

[workers]
node2 ansible_host=192.168.1.12
node3 ansible_host=192.168.1.13

[all:vars]
ansible_user=${ANSIBLE_USER}
ansible_ssh_private_key_file=${SSH_KEY}
ansible_python_interpreter=/usr/bin/python3
ansible_ssh_common_args=-o StrictHostKeyChecking=no

[workers:vars]
ansible_ssh_common_args=-o StrictHostKeyChecking=no \
  -o ProxyCommand="ssh -i ${SSH_KEY} -o StrictHostKeyChecking=no -W %h:%p ${ANSIBLE_USER}@${CONTROL_FIP}"
EOF

echo "Inventory written:"
cat "$ANSIBLE_DIR/inventory.ini"

echo -e "\n>>> 4. Bootstrap full Python & six on every node (raw module)"
export PATH="$HOME/.local/bin:$PATH"
ANSIBLE_HOST_KEY_CHECKING=False \
ansible -i "$ANSIBLE_DIR/inventory.ini" all \
  -m raw \
  -a 'sudo DEBIAN_FRONTEND=noninteractive apt-get update -y &&
      sudo DEBIAN_FRONTEND=noninteractive apt-get install -y python3 python3-apt python3-pip python3-six python3-setuptools' \
  --become -vv

echo -e "\n>>> 5. Connectivity test (ansible ping)"
ANSIBLE_HOST_KEY_CHECKING=False \
ansible -i "$ANSIBLE_DIR/inventory.ini" all -m ping

##########################################################################
# 6. RUN THE PROFESSOR'S PLAYBOOKS IN ORDER
##########################################################################
cd "$ANSIBLE_DIR"

run_pb () {
  local pb=$1
  echo -e "\n─── Running $pb ───"
  ansible-playbook -i inventory.ini "$pb"
}

run_pb general.yml
run_pb pre_k8s.yml
run_pb k8s/cluster.yml
run_pb post_k8s.yml
run_pb argocd/add_platform.yml
run_pb argocd/add_staging.yml
run_pb argocd/add_prod.yml
run_pb argocd/add_canary.yml
run_pb argocd/workflow_templates_apply.yml

echo -e "\n✅  All professor playbooks completed successfully!"



>>> 1. Install/upgrade Ansible locally (if missing)
ansible 2.10.17

>>> 2. Fetch node1 floating IP from Terraform output
Control node public IP: 129.114.26.17

>>> 3. Build inventory.ini (with ProxyCommand for workers)
Inventory written:
[control]
node1 ansible_host=129.114.26.17

[workers]
node2 ansible_host=192.168.1.12
node3 ansible_host=192.168.1.13

[all:vars]
ansible_user=cc
ansible_ssh_private_key_file=/home/ng3230_nyu_edu/.ssh/id_rsa
ansible_python_interpreter=/usr/bin/python3
ansible_ssh_common_args=-o StrictHostKeyChecking=no

[workers:vars]
ansible_ssh_common_args=-o StrictHostKeyChecking=no   -o ProxyCommand="ssh -i /home/ng3230_nyu_edu/.ssh/id_rsa -o StrictHostKeyChecking=no -W %h:%p cc@129.114.26.17"

>>> 4. Bootstrap full Python & six on every node (raw module)
ansible 2.10.17
  config file = /work/MLops-Final-Project/infrastructure/ansible/ansible.cfg
  configured module search path = ['/home/ng3230_nyu_edu/.ansible/plugins/modules', '/usr/share/ansible/plugins/module

In [54]:
# %%bash
ANSIBLE_DIR=/work/MLops-Final-Project/infrastructure/ansible
export PATH=$HOME/.local/bin:$PATH
ANSIBLE_HOST_KEY_CHECKING=False

echo "Installing ansible-core, python3-six on all nodes …"
ansible -i "$ANSIBLE_DIR/inventory.ini" all \
  -m raw \
  -a 'sudo DEBIAN_FRONTEND=noninteractive apt-get update -y &&
      sudo DEBIAN_FRONTEND=noninteractive apt-get install -y ansible-core python3-six python3-setuptools' \
  --become -vv


bash: fg: %%bash: no such job
Installing ansible-core, python3-six on all nodes …
ansible 2.10.17
  config file = None
  configured module search path = ['/home/ng3230_nyu_edu/.ansible/plugins/modules', '/usr/share/ansible/plugins/modules']
  ansible python module location = /opt/conda/lib/python3.10/site-packages/ansible
  executable location = /opt/conda/bin/ansible
  python version = 3.10.9 | packaged by conda-forge | (main, Feb  2 2023, 20:20:04) [GCC 11.3.0]
No config file found; using defaults
Skipping callback 'default', as we already have a stdout callback.
Skipping callback 'minimal', as we already have a stdout callback.
Skipping callback 'oneline', as we already have a stdout callback.
META: ran handlers
node1 | CHANGED | rc=0 >>
Get:1 http://nova.clouds.archive.ubuntu.com/ubuntu noble InRelease [256 kB]
Hit:2 http://security.ubuntu.com/ubuntu noble-security InRelease               
Get:3 http://nova.clouds.archive.ubuntu.com/ubuntu noble-updates InRelease [126 kB]
Get:4 htt

In [55]:
ansible -i "$ANSIBLE_DIR/inventory.ini" all -m ping


An exception occurred during task execution. To see the full traceback, use -vvv. The error was: ModuleNotFoundError: No module named 'ansible.module_utils.six.moves'
node1 | FAILED! => {
    "changed": false,
    "module_stderr": "Shared connection to 129.114.26.17 closed.\r\n",
    "module_stdout": "Traceback (most recent call last):\r\n  File \"/home/cc/.ansible/tmp/ansible-tmp-1746752500.4160743-6605-157766089529624/AnsiballZ_ping.py\", line 102, in <module>\r\n    _ansiballz_main()\r\n  File \"/home/cc/.ansible/tmp/ansible-tmp-1746752500.4160743-6605-157766089529624/AnsiballZ_ping.py\", line 94, in _ansiballz_main\r\n    invoke_module(zipped_mod, temp_path, ANSIBALLZ_PARAMS)\r\n  File \"/home/cc/.ansible/tmp/ansible-tmp-1746752500.4160743-6605-157766089529624/AnsiballZ_ping.py\", line 37, in invoke_module\r\n    from ansible.module_utils import basic\r\n  File \"/tmp/ansible_ping_payload_25u3930_/ansible_ping_payload.zip/ansible/module_utils/basic.py\", line 176, in <module>\r\nMo

: 2

In [56]:
# %%bash
ANSIBLE_DIR=/work/MLops-Final-Project/infrastructure/ansible
export PATH=$HOME/.local/bin:$PATH

echo "1) Which Python is Ansible using?"
ansible -i $ANSIBLE_DIR/inventory.ini all -m raw -a 'echo $PATH && which python3' -o

echo -e "\n2) Does the ansible-core package exist?"
ansible -i $ANSIBLE_DIR/inventory.ini all -m raw -a 'dpkg -l ansible-core | grep ^ii || echo "NOT INSTALLED"' -o

echo -e "\n3) Can Python import ansible.module_utils.six ?"
ansible -i $ANSIBLE_DIR/inventory.ini all -m raw -a 'python3 - <<PY\nimport importlib, sys\ntry:\n import ansible.module_utils.six as s\n print(\"OK\")\nexcept ImportError as e:\n print(\"FAIL\", e)\n sys.exit(1)\nPY' -o

echo -e "\n4) List ansible.module_utils dir to see what’s there"
ansible -i $ANSIBLE_DIR/inventory.ini all -m raw -a 'python3 -c "import pkg_resources, os, sys, importlib.util as iu; p=iu.find_spec(\"ansible.module_utils\").submodule_search_locations[0]; print(p); print(os.listdir(p))"' -o


1) Which Python is Ansible using?
node1 | CHANGED | rc=0 | (stdout) /usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin\r\n/usr/bin/python3\r\n (stderr) Shared connection to 129.114.26.17 closed.\r\n
node2 | CHANGED | rc=0 | (stdout) /usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin\r\n/usr/bin/python3\r\n (stderr) Shared connection to 192.168.1.12 closed.\r\n
node3 | CHANGED | rc=0 | (stdout) /usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin\r\n/usr/bin/python3\r\n (stderr) Shared connection to 192.168.1.13 closed.\r\n

2) Does the ansible-core package exist?
node3 | CHANGED | rc=0 | (stdout) ii  ansible-core   2.16.3-0ubuntu2 all          Configuration management, deployment, and task execution system\r\n (stderr) Shared connection to 192.168.1.13 closed.\r\n
node1 | CHANGED | rc=0 | (stdout) ii  ansible-core   2.16.3-0ubuntu2 all          C

In [57]:
# %%bash
ANSIBLE_DIR=/work/MLops-Final-Project/infrastructure/ansible
export PATH=$HOME/.local/bin:$PATH
ANSIBLE_HOST_KEY_CHECKING=False

echo "== Before =="
ansible -i $ANSIBLE_DIR/inventory.ini all \
  -m raw -a 'dpkg -l python3-six ansible-core | grep ^ii || true' -o

echo -e "\n== Installing python3-six & ansible-core =="
ansible -i $ANSIBLE_DIR/inventory.ini all \
  -m raw \
  -a 'sudo DEBIAN_FRONTEND=noninteractive apt-get update -y &&
      sudo DEBIAN_FRONTEND=noninteractive apt-get install -y python3-six ansible-core' \
  --become -o

echo -e "\n== After =="
ansible -i $ANSIBLE_DIR/inventory.ini all \
  -m raw -a 'dpkg -l python3-six ansible-core | grep ^ii' -o

echo -e "\n== Ping test =="
ANSIBLE_HOST_KEY_CHECKING=False \
ansible -i $ANSIBLE_DIR/inventory.ini all -m ping -o


== Before ==
node1 | CHANGED | rc=0 | (stdout) ii  ansible-core   2.16.3-0ubuntu2 all          Configuration management, deployment, and task execution system\r\nii  python3-six    1.16.0-4        all          Python 2 and 3 compatibility library\r\n (stderr) Shared connection to 129.114.26.17 closed.\r\n
node2 | CHANGED | rc=0 | (stdout) ii  ansible-core   2.16.3-0ubuntu2 all          Configuration management, deployment, and task execution system\r\nii  python3-six    1.16.0-4        all          Python 2 and 3 compatibility library\r\n (stderr) Shared connection to 192.168.1.12 closed.\r\n
node3 | CHANGED | rc=0 | (stdout) ii  ansible-core   2.16.3-0ubuntu2 all          Configuration management, deployment, and task execution system\r\nii  python3-six    1.16.0-4        all          Python 2 and 3 compatibility library\r\n (stderr) Shared connection to 192.168.1.13 closed.\r\n

== Installing python3-six & ansible-core ==
node3 | CHANGED | rc=0 | (stdout) \r0% [Working]\r            

: 2

In [2]:
# 1) Remove the apt copy (keeps only python libs on the targets)
sudo apt-get remove -y ansible-core

# 2) Install 2.15.x in your user env (control plane only)
pip install --user --upgrade "ansible-core<2.16"   # installs 2.15.5 today
export PATH=$HOME/.local/bin:$PATH
ansible --version    # should say 2.15.x


Reading package lists... 0%Reading package lists... 0%Reading package lists... 0%Reading package lists... 4%Reading package lists... 4%Reading package lists... 4%Reading package lists... 4%Reading package lists... 17%Reading package lists... 40%Reading package lists... 45%Reading package lists... 45%Reading package lists... 46%Reading package lists... 46%Reading package lists... 55%Reading package lists... 55%Reading package lists... 58%Reading package lists... 68%Reading package lists... 68%Reading package lists... 73%Reading package lists... 73%Reading package lists... 73%Reading package lists... 73%Reading package lists... 73%Reading package lists... 73%Reading package lists... 73%Reading package lists... 73%Reading package lists... 82%Reading package lists... 82%Reading package lists... 82%Reading package lists... 94%Reading package lists... 94%Reading package lists... 98%Reading package lists... 98%Reading package lists... 98%Reading package lists... 98%Reading package lists... 99

In [3]:
ANSIBLE_HOST_KEY_CHECKING=False \
ansible -i /work/MLops-Final-Project/infrastructure/ansible/inventory.ini \
       all -m ping


node1 | SUCCESS => {
    "changed": false,
    "ping": "pong"
}
node2 | SUCCESS => {
    "changed": false,
    "ping": "pong"
}
node3 | SUCCESS => {
    "changed": false,
    "ping": "pong"
}


In [6]:
export ANSIBLE_CONFIG=/work/MLops-Final-Project/infrastructure/ansible/ansible.cfg
cd /work/MLops-Final-Project/infrastructure/ansible

# fast check
ansible all -i inventory.ini -m ping

# full run
ansible-playbook -i inventory.ini site.yml


node1 | SUCCESS => {
    "changed": false,
    "ping": "pong"
}
node3 | SUCCESS => {
    "changed": false,
    "ping": "pong"
}
node2 | SUCCESS => {
    "changed": false,
    "ping": "pong"
}
ERROR! Unable to retrieve file contents
Could not find or access '/work/MLops-Final-Project/infrastructure/ansible/k8s/inventory/mycluster/hosts.yml' on the Ansible Controller.
If you are using a module and expect the file to exist on the remote, see the remote_src option


: 1